In [1]:
import os
import re
from pathlib import Path
import pandas as pd
import numpy as np





In [2]:
# import pandas as pd
# import sys
#
# # --- Configuration ---
#
# # The CSV file you uploaded
# INPUT_CSV_FILE = 'data/final_merged_fmri_t1_clinical (2).csv'
#
# # The new Excel file we will create
# OUTPUT_EXCEL_FILE = 'data/T1_first_session_data.xlsx'
#
# # The one column you want to include in EVERY sheet
# # Based on your request, this is 'Subject_Code'
# KEY_COLUMN = 'Subject_Code'
#
# # VVV --- THIS IS THE PART YOU EDIT --- VVV
# #
# # Define the sheets you want to create.
# #
# # - The 'key' is the name you want for the sheet (e.g., 'Clinical_Scores').
# # - The 'value' is a list of all the *other* columns you want in that sheet.
# #
# # The KEY_COLUMN ('Subject_Code') will be added for you automatically.
# # I've added some examples based on your CSV's column names.
# #
# SHEETS_TO_CREATE = {
#     'global_parameters': [
#     "brainsegvol",
#     "cerebralwhitemattervol",
#     "cortexvol",
#     "etiv",
#     "lhcortexvol",
#     "rhcortexvol",
#     "subcortgrayvol",
#     "totalgrayvol",
#     "meanthickness_lh",
#     "meanthickness_rh",
#     "whitesurfarea_lh",
#     "whitesurfarea_rh"],
#
#     'subcortical_vol': [
#     "3rd_ventricle",
#     "4th_ventricle",
#     "5th_ventricle",
#     "brain_stem",
#     "cc_anterior",
#     "cc_central",
#     "cc_mid_anterior",
#     "cc_mid_posterior",
#     "cc_posterior",
#     "csf",
#     "left_accumbens_area",
#     "left_amygdala",
#     "left_caudate",
#     "left_cerebellum_cortex",
#     "left_cerebellum_white_matter",
#     "left_choroid_plexus",
#     "left_hippocampus",
#     "left_inf_lat_vent",
#     "left_lateral_ventricle",
#     "left_non_wm_hypointensities",
#     "left_pallidum",
#     "left_putamen",
#     "left_thalamus",
#     "left_ventraldc",
#     "left_vessel",
#     "left_wm_hypointensities",
#     "non_wm_hypointensities",
#     "optic_chiasm",
#     "right_accumbens_area",
#     "right_amygdala",
#     "right_caudate",
#     "right_cerebellum_cortex",
#     "right_cerebellum_white_matter",
#     "right_choroid_plexus",
#     "right_hippocampus",
#     "right_inf_lat_vent",
#     "right_lateral_ventricle",
#     "right_non_wm_hypointensities",
#     "right_pallidum",
#     "right_putamen",
#     "right_thalamus",
#     "right_ventraldc",
#     "right_vessel",
#     "right_wm_hypointensities",
#     "wm_hypointensities"
# ],
#
#     "cortical_vol_lr": [
#     "bankssts_lh",
#     "bankssts_rh",
#     "caudalanteriorcingulate_lh",
#     "caudalanteriorcingulate_rh",
#     "caudalmiddlefrontal_lh",
#     "caudalmiddlefrontal_rh",
#     "cuneus_lh",
#     "cuneus_rh",
#     "entorhinal_lh",
#     "entorhinal_rh",
#     "frontalpole_lh",
#     "frontalpole_rh",
#     "fusiform_lh",
#     "fusiform_rh",
#     "inferiorparietal_lh",
#     "inferiorparietal_rh",
#     "inferiortemporal_lh",
#     "inferiortemporal_rh",
#     "insula_lh",
#     "insula_rh",
#     "isthmuscingulate_lh",
#     "isthmuscingulate_rh",
#     "lateraloccipital_lh",
#     "lateraloccipital_rh",
#     "lateralorbitofrontal_lh",
#     "lateralorbitofrontal_rh",
#     "lingual_lh",
#     "lingual_rh",
#     "medialorbitofrontal_lh",
#     "medialorbitofrontal_rh",
#     "middletemporal_lh",
#     "middletemporal_rh",
#     "paracentral_lh",
#     "paracentral_rh",
#     "parahippocampal_lh",
#     "parahippocampal_rh",
#     "parsopercularis_lh",
#     "parsopercularis_rh",
#     "parsorbitalis_lh",
#     "parsorbitalis_rh",
#     "parstriangularis_lh",
#     "parstriangularis_rh",
#     "pericalcarine_lh",
#     "pericalcarine_rh",
#     "postcentral_lh",
#     "postcentral_rh",
#     "posteriorcingulate_lh",
#     "posteriorcingulate_rh",
#     "precentral_lh",
#     "precentral_rh",
#     "precuneus_lh",
#     "precuneus_rh",
#     "rostralanteriorcingulate_lh",
#     "rostralanteriorcingulate_rh",
#     "rostralmiddlefrontal_lh",
#     "rostralmiddlefrontal_rh",
#     "superiorfrontal_lh",
#     "superiorfrontal_rh",
#     "superiorparietal_lh",
#     "superiorparietal_rh",
#     "superiortemporal_lh",
#     "superiortemporal_rh",
#     "supramarginal_lh",
#     "supramarginal_rh",
#     "temporalpole_lh",
#     "temporalpole_rh",
#     "transversetemporal_lh",
#     "transversetemporal_rh"
# ]
# ,
#     'cortical_lobes_lr': [
#     "Cingulate_lh",
#     "Cingulate_rh",
#     "Frontal_lh",
#     "Frontal_rh",
#     "Insula_lh",
#     "Insula_rh",
#     "Occipital_lh",
#     "Occipital_rh",
#     "Parietal_lh",
#     "Parietal_rh",
#     "Temporal_lh",
#     "Temporal_rh"
# ],
#
#
#     'cortical_lobes_full_brain': [
#     "cingulate",
#     "frontal",
#     "insula",
#     "occipital",
#     "parietal",
#     "temporal"
# ],
#     'cortical_networks_full_brain': [
#     "DMN_GM",
#     "DA_GM",
#     "FPN_GM",
#     "LIM_GM",
#     "SMN_GM",
#     "VAN_GM",
#     "VIS_GM"
# ]
#
# }
# # ^^^ --- THIS IS THE PART YOU EDIT --- ^^^
#
#


In [3]:
# """
# Main function to read CSV and write to a multi-sheet Excel file.
# """
# try:
#     # 1. Load the entire CSV into a pandas DataFrame
#     print(f"Reading input file: {INPUT_CSV_FILE}...")
#     main_df = pd.read_csv(INPUT_CSV_FILE)
#     print("CSV file loaded successfully.")
#
#     # Build a case-insensitive mapping: lower_name -> original_name
#     col_map = {c.lower(): c for c in main_df.columns}
#
#     # 2. Check that the key column exists (case-insensitive)
#     key_lower = KEY_COLUMN.lower()
#     if key_lower not in col_map:
#         print(f"Error: The key column '{KEY_COLUMN}' was not found in the CSV (case-insensitive search).")
#         print("Please check the KEY_COLUMN variable.")
#         print(f"Available columns are: {main_df.columns.tolist()}")
#         sys.exit(1)
#
#     key_actual = col_map[key_lower]  # actual column name in the DataFrame
#
#     # 3. Create an ExcelWriter object to handle writing to multiple sheets
#     with pd.ExcelWriter(OUTPUT_EXCEL_FILE, engine='openpyxl') as writer:
#         print(f"Creating Excel file: {OUTPUT_EXCEL_FILE}...")
#
#         # 4. Loop through your configuration
#         for sheet_name, column_list in SHEETS_TO_CREATE.items():
#
#             # 5. Build the final list of columns for this sheet
#             final_columns = []
#
#             # Check if key column is already in the list (case-insensitive)
#             column_list_lower = [c.lower() for c in column_list]
#             if key_lower not in column_list_lower:
#                 final_columns.append(KEY_COLUMN)   # we'll resolve to real name below
#
#             final_columns.extend(column_list)
#
#             # 6. Check for missing columns and create the sheet DataFrame
#             valid_columns = []
#             seen_lowers = set()  # to avoid duplicates with different cases
#
#             for col in final_columns:
#                 col_lower = col.lower()
#                 if col_lower in seen_lowers:
#                     continue  # skip duplicates
#                 if col_lower in col_map:
#                     valid_columns.append(col_map[col_lower])  # use actual name from CSV
#                     seen_lowers.add(col_lower)
#                 else:
#                     print(f"  - Warning: Column '{col}' (for sheet '{sheet_name}') "
#                           f"not found in CSV (case-insensitive). It will be skipped.")
#
#             if not valid_columns:
#                 print(f"  - Error: No valid columns found for sheet '{sheet_name}'. "
#                       f"This sheet will be empty or skipped.")
#                 continue
#
#             # 7. Create the new DataFrame for this sheet
#             sheet_df = main_df[valid_columns]
#
#             # 8. Write this DataFrame to the Excel file
#             sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)
#             print(f"  -> Successfully created sheet: '{sheet_name}' with {len(valid_columns)} columns.")
#
#     print(f"\nAll done! Your file '{OUTPUT_EXCEL_FILE}' has been created.")
#
# except FileNotFoundError:
#     print(f"Error: The file '{INPUT_CSV_FILE}' was not found.")
#     print("Please make sure the CSV file is in the same directory as this script.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")


# functions

In [4]:
def collect_files(folder: Path, recursive: bool = False):
    if recursive:
        return [p for p in folder.rglob("*.xlsx") if p.is_file()]
    return [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() == ".xlsx"]


In [5]:

def list_excel_files(folder: Path, recursive: bool = False):
    if recursive:
        return sorted([p for p in folder.rglob("*.xlsx") if p.is_file()])
    return sorted([p for p in folder.iterdir() if p.is_file() and p.suffix.lower() == ".xlsx"])



In [6]:
def pick_column(df: pd.DataFrame, candidates):
    """Return the first matching column name in df (case-insensitive), else None."""
    cols_map = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in cols_map:
            return cols_map[c.lower()]
    # try partial matches (e.g., "grayvol" inside "grayvol (mm3)")
    for c in candidates:
        for k, v in cols_map.items():
            if c.lower() in k:
                return v
    return None


In [7]:
def slugify(name: str) -> str:
    """lowercase + replace non-alnum with underscores + collapse repeats."""
    s = re.sub(r"\W+", "_", str(name).strip().lower())
    s = re.sub(r"_+", "_", s).strip("_")
    return s

In [8]:
# Create a stable, pretty column order:
#   subject_code first, then all lh/rh ses1 followed by ses2 (alphabetical by struct)
def col_sort_key(c):
    # subject_code stays first
    if c == "subject_code":
        return (0, "", "", 0)
    # parse pattern: <struct>_<hemi>_ses<1|2>
    m = re.match(r"^(.*)_(lh|rh)_ses([12])$", c)
    if m:
        struct, hemi, ses = m.group(1), m.group(2), int(m.group(3))
        return (1, struct, hemi, ses)
    # anything unexpected goes to the end
    return (2, c, "", 99)

In [9]:
def merge_two_cols(dframe, cols, how="sum"):
    # cols is a list of existing columns (1 or 2). We want:
    # - NaN if both missing
    # - sum/mean ignoring NaNs otherwise
    cols = [c for c in cols if c is not None and c in dframe.columns]
    if not cols:
        return pd.Series(np.nan, index=dframe.index)
    if how == "mean":
        return dframe[cols].mean(axis=1)
    # default sum
    return dframe[cols].sum(axis=1, min_count=1)


In [10]:
def reduce_cols(dframe, cols, how="sum"):
    cols = [c for c in cols if c in dframe.columns]
    if not cols:
        return pd.Series(np.nan, index=dframe.index)
    return dframe[cols].sum(axis=1, min_count=1)

In [11]:
# --- 4) Order columns nicely: subject_code, then lobe alpha, ses1 before ses2, lh before rh
def order_key_lobe(c):
    if c == "subject_code":
        return (0, "", 0, "")
    m = re.match(r"^(?P<lobe>.+)_(?P<hemi>lh|rh)_ses(?P<ses>\d+)$", c)
    if m:
        lobe = m.group("lobe")
        hemi = m.group("hemi")
        ses  = int(m.group("ses"))
        hemi_order = 0 if hemi == "lh" else 1
        return (1, lobe, ses, hemi_order)
    return (2, c, 99, 99)


In [12]:
def _read_csv(path: Path, merge_key: str) -> pd.DataFrame:
    if not path.exists():
        print(f"WARNING: file not found -> {path}")
        return pd.DataFrame(columns=[merge_key])
    df = pd.read_csv(path)
    # normalize key dtype for safe merging/sorting
    if merge_key in df.columns:
        df[merge_key] = df[merge_key].astype(str)
    return df



In [13]:
def _order_columns_subject_first(df: pd.DataFrame, key: str = "subject_code") -> pd.DataFrame:
    cols = list(df.columns)
    if key in cols:
        return df[[key] + [c for c in cols if c != key]]
    return df

In [14]:
def _sort_consistent(df: pd.DataFrame, key: str = "subject_code") -> pd.DataFrame:
    """Alphabetical by key; if numbers exist, use them as a secondary natural sort."""
    if key not in df.columns:
        return df
    key_series = df[key].astype(str)
    # Extract first number if present; fall back to NaN
    num = key_series.str.extract(r"(\d+)")[0].astype(float)
    # Primary: alphabetical by key; Secondary: numeric ascending (NaN last)
    return (df.assign(_num=num)
              .sort_values(by=[key, "_num"], kind="mergesort")
              .drop(columns="_num"))


In [15]:
def _autosize_columns(writer: pd.ExcelWriter, df: pd.DataFrame, sheet_name: str, max_width: int = 60):
    """Best-effort autosize; harmless no-op if engine doesn't support."""
    try:
        ws = writer.sheets[sheet_name]
        for idx, col in enumerate(df.columns, 1):
            s = df[col].astype(str)
            longest = max([len(str(col))] + [len(x) for x in s.tolist()] + [10])
            ws.column_dimensions[ws.cell(row=1, column=idx).column_letter].width = min(longest + 2, max_width)
    except Exception:
        pass

In [16]:
records = []
sheet_names_ref = None
# --- CONFIG --- #
FOLDER = Path("data/stats_t1_fs")  # change to your folder
SUMMARY_CSV = Path("data/stats_t1_fs/ses1_and_ses2_T1_summary.csv")  # change if you want
RECURSIVE = False  # set True if files are in subfolders
# --------------- #


# first session

## extracting from the subject file the cortical volumes and merge nto one csv file

In [17]:

# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")  # change to your folder
OUT_CSV = Path("data/stats_t1_fs/ses1/ses1_aparc_voxels_lh_rh.csv")  # output file
SHEET_NAME = "aparc_voxels"  # sheet to read
RECURSIVE = False            # set True if files are in subfolders
# ----------------------------

file_regex = re.compile(
    r"sub-(?P<subject>\d{3})_(?P<group>CT|NT)_ses-(?P<session>[12])_merged\.xlsx$",
    re.IGNORECASE,
)

# Accumulate rows keyed by subject_code
rows_by_subject = {}
all_feature_cols = set()
files = list_excel_files(FOLDER, RECURSIVE)
for path in files:
    print("FULL PATH :", repr(str(path)))
    print("NAME ONLY :", repr(path.name))
    m = file_regex.match(path.name)
    print(m)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))
    print(session)

    # --- ONLY PROCESS SESSION 1 ---
    if session != 1:
        continue

    subject_code = f"{group}{subject}"

    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    # Locate columns
    col_struct = pick_column(df, ["StructName"])
    col_hemi   = pick_column(df, ["Hemisphere"])
    col_gray   = pick_column(df, ["GrayVol"])

    if not all([col_struct, col_hemi, col_gray]):
        print(f"Warning: missing needed columns in {path.name}. "
              f"StructName={col_struct}, Hemisphere={col_hemi}, GrayVol={col_gray}")
        continue

    # Initialize row
    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # Process rows
    for _, row in df.iterrows():
        struct = slugify(row[col_struct])
        hemi = slugify(row[col_hemi])

        if hemi in ("left", "l"): hemi = "lh"
        if hemi in ("right", "r"): hemi = "rh"

        if not struct or hemi not in ("lh", "rh"):
            continue

        # --- NEW: COLUMN NAME WITHOUT SESSION ---
        col_name = f"{struct}_{hemi}"

        # Convert to numeric
        val = row[col_gray]
        try:
            val = pd.to_numeric(val)
        except:
            val = np.nan

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)

# Build DataFrame
df_out = pd.DataFrame(rows_by_subject.values())
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan
df_out = df_out[all_columns]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(5))



FULL PATH : 'data\\stats_t1_fs\\mapping_lobes_networks.xlsx'
NAME ONLY : 'mapping_lobes_networks.xlsx'
None
FULL PATH : 'data\\stats_t1_fs\\sub-002_CT_ses-1_merged.xlsx'
NAME ONLY : 'sub-002_CT_ses-1_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-002_CT_ses-1_merged.xlsx'>
1
FULL PATH : 'data\\stats_t1_fs\\sub-002_NT_ses-1_merged.xlsx'
NAME ONLY : 'sub-002_NT_ses-1_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-002_NT_ses-1_merged.xlsx'>
1
FULL PATH : 'data\\stats_t1_fs\\sub-002_NT_ses-2_merged.xlsx'
NAME ONLY : 'sub-002_NT_ses-2_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-002_NT_ses-2_merged.xlsx'>
2
FULL PATH : 'data\\stats_t1_fs\\sub-003_CT_ses-1_merged.xlsx'
NAME ONLY : 'sub-003_CT_ses-1_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-003_CT_ses-1_merged.xlsx'>
1
FULL PATH : 'data\\stats_t1_fs\\sub-003_CT_ses-2_merged.xlsx'
NAME ONLY : 'sub-003_CT_ses-2_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-003_CT_ses-2_merged.xlsx'>
2
FULL PA

,subject_code,bankssts_lh,bankssts_rh,caudalanteriorcingulate_lh,caudalanteriorcingulate_rh,caudalmiddlefrontal_lh,caudalmiddlefrontal_rh,cuneus_lh,cuneus_rh,entorhinal_lh,...,superiorparietal_lh,superiorparietal_rh,superiortemporal_lh,superiortemporal_rh,supramarginal_lh,supramarginal_rh,temporalpole_lh,temporalpole_rh,transversetemporal_lh,transversetemporal_rh
0,CT002,3247,2390,1593,2232,6888,6822,2763,3254,1433,...,16606,16481,15329,14010,14012,11521,1484,1787,1362,904
1,NT002,2598,1986,1424,2187,5876,6710,3117,5185,1679,...,13418,14705,12423,12143,10160,10382,2314,2215,971,781
2,CT003,2650,2683,1655,1911,7284,6086,3045,3354,2361,...,15518,15468,12005,12525,9350,9664,3098,3175,1042,776
3,NT003,2628,2827,2389,2589,7100,7939,2964,3984,2048,...,15089,15467,15034,13084,14066,12609,2633,2607,1289,883
4,CT004,2598,2386,2530,2906,8093,7395,2834,3300,1767,...,15353,14536,14995,13529,11490,10928,2404,2433,1470,1058


## extracting from the subject file the subcortical volumes and merge nto one csv file


In [18]:
# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")
OUT_CSV = Path("data/stats_t1_fs/ses1/ses1_aseg_data.csv")
SHEET_NAME = "aseg_voxels"
RECURSIVE = False
# ----------------------------

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, RECURSIVE)
for path in files:

    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    # --- ONLY USE SESSION 1 ---
    if session != 1:
        continue

    subject_code = f"{group}{subject}"

    # read the aseg sheet
    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    # locate required columns
    col_struct = pick_column(df, ["StructName"])
    col_gray   = pick_column(df, ["Volume_mm3"])

    if not all([col_struct, col_gray]):
        print(f"Warning: missing needed columns in {path.name}. struct={col_struct}, vol={col_gray}")
        continue

    # initialize subject row
    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # iterate rows
    for _, row in df.iterrows():
        struct = slugify(row[col_struct])
        if not struct:
            continue

        # --- FIX: NO SESSION SUFFIX ---
        col_name = struct

        val = row[col_gray]
        try:
            val = pd.to_numeric(val)
        except:
            val = np.nan

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

    # optional logging


# final table columns
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)

# build wide dataframe
df_out = pd.DataFrame(rows_by_subject.values())

# ensure missing columns exist
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan

df_out = df_out[all_columns]

# write CSV
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(5))


Saved: data\stats_t1_fs\ses1\ses1_aseg_data.csv


,subject_code,3rd_ventricle,4th_ventricle,5th_ventricle,brain_stem,cc_anterior,cc_central,cc_mid_anterior,cc_mid_posterior,cc_posterior,...,right_inf_lat_vent,right_lateral_ventricle,right_non_wm_hypointensities,right_pallidum,right_putamen,right_thalamus,right_ventraldc,right_vessel,right_wm_hypointensities,wm_hypointensities
0,CT002,946.8,1479.5,0.0,20198.9,838.9,395.9,462.1,490.1,762.6,...,258.1,7300.3,0.0,1714.5,4504.4,7452.5,3682.2,13.3,0.0,627.2
1,NT002,933.1,981.6,0.0,19351.4,703.9,634.9,502.0,387.5,894.1,...,232.4,3594.3,0.0,1860.7,4944.3,6697.8,3928.0,19.4,0.0,654.5
2,CT003,795.9,1651.5,0.0,20283.0,869.6,700.3,607.8,571.3,998.6,...,356.7,5403.1,0.0,1959.3,4668.5,7696.7,3794.7,15.0,0.0,661.7
3,NT003,771.7,1849.9,0.0,23059.0,977.6,777.6,633.0,612.8,1137.3,...,293.5,5516.4,0.0,2161.4,4955.2,8696.2,4256.2,7.8,0.0,758.7
4,CT004,1019.4,2162.6,0.0,19347.2,1079.6,512.7,478.8,638.7,1027.5,...,386.2,9159.8,0.0,1898.0,5026.0,7783.8,4164.7,10.8,0.0,487.7


## extracting from the subject file thewhite matter  volumes and merge nto one csv file


In [19]:
# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")
OUT_CSV = Path("data/stats_t1_fs/ses1/ses1_wmparc_data.csv")
SHEET_NAME = "wmparc_voxels"
RECURSIVE = False
# ----------------------------

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, RECURSIVE)
for path in files:

    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    # --- ONLY USE SESSION 1 ---
    if session != 1:
        continue

    subject_code = f"{group}{subject}"

    # Load wmparc sheet
    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    # Identify needed columns
    col_struct = pick_column(df, ["StructName"])
    col_gray   = pick_column(df, ["Volume_mm3"])

    if not all([col_struct, col_gray]):
        print(f"Warning: missing needed columns in {path.name}. struct={col_struct}, vol={col_gray}")
        continue

    # Initialize subject row
    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # Process each ROI row
    for _, row in df.iterrows():
        struct = slugify(row[col_struct])
        if not struct:
            continue

        # --- FIX: remove session suffix ---
        col_name = struct

        # convert to numeric
        val = row[col_gray]
        try:
            val = pd.to_numeric(val)
        except:
            val = np.nan

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

    # optional tracking

# Final set of columns
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)

# Build final table
df_out = pd.DataFrame(rows_by_subject.values())

# Ensure all expected columns exist
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan

df_out = df_out[all_columns]

# Save CSV
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(5))


Saved: data\stats_t1_fs\ses1\ses1_wmparc_data.csv


,subject_code,left_unsegmentedwhitematter,right_unsegmentedwhitematter,wm_lh_bankssts,wm_lh_caudalanteriorcingulate,wm_lh_caudalmiddlefrontal,wm_lh_cuneus,wm_lh_entorhinal,wm_lh_frontalpole,wm_lh_fusiform,...,wm_rh_precentral,wm_rh_precuneus,wm_rh_rostralanteriorcingulate,wm_rh_rostralmiddlefrontal,wm_rh_superiorfrontal,wm_rh_superiorparietal,wm_rh_superiortemporal,wm_rh_supramarginal,wm_rh_temporalpole,wm_rh_transversetemporal
0,CT002,24819.4,25086.5,3408.9,3590.1,6647.4,1817.8,647.1,259.0,7629.1,...,13317.3,10986.8,2228.7,11952.3,17967.1,11795.6,7314.8,8828.6,557.5,699.3
1,NT002,28695.5,29649.8,3256.2,2437.9,5984.8,3093.4,728.1,332.9,5908.8,...,12051.8,7657.1,1736.9,11152.4,15386.1,11491.3,6462.8,7443.3,623.1,464.3
2,CT003,33322.6,31838.1,4073.6,3197.4,7118.4,2322.3,1036.5,261.5,7846.3,...,14465.7,10235.0,1812.8,12669.7,16985.7,12255.8,5995.6,8215.8,646.1,560.0
3,NT003,32881.1,31335.8,3774.9,4236.0,6450.3,2498.9,1219.6,279.3,6560.0,...,13129.2,10774.6,1690.9,16402.2,19051.9,13307.8,7059.9,9943.8,756.8,758.6
4,CT004,27175.7,26310.7,2227.5,3318.3,7673.4,2376.6,912.4,376.0,5927.0,...,14148.8,10280.0,1896.0,15336.4,21204.6,11449.5,6947.3,9068.5,820.7,664.1


## ecxtrating from the cortical parameters of each subject into one csv

In [20]:
# Build per-subject table from aparc_summary, including hemisphere in column names.
# Requires helpers already defined: file_regex, list_excel_files, slugify, pick_column, col_sort_key, mark_extraction_complete

from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")
OUT_CSV = Path("data/stats_t1_fs/ses1/ses1_aparc_summary_data.csv")
SHEET_NAME = "aparc_summary"

SHORTNAME_COLUMN_CANDIDATES = ["ShortName"]
VALUE_COLUMN_CANDIDATES     = ["Value"]
HEMI_COLUMN_CANDIDATES      = ["Hemisphere"]

SHORTNAMES_TO_KEEP = ["WhiteSurfArea", "MeanThickness"]

RENAMES = {}
# ----------------------------

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, recursive=False)
for path in files:

    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    # --- ONLY SESSION 1 ---
    if session != 1:
        continue

    subject_code = f"{group}{subject}"

    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    col_short = pick_column(df, SHORTNAME_COLUMN_CANDIDATES)
    col_val   = pick_column(df, VALUE_COLUMN_CANDIDATES)
    col_hemi  = pick_column(df, HEMI_COLUMN_CANDIDATES)

    if not all([col_short, col_val, col_hemi]):
        print(f"Warning: missing needed columns in {path.name}. "
              f"ShortName={col_short}, Value={col_val}, Hemisphere={col_hemi}")
        continue

    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # keep only desired ShortNames
    sub = df[df[col_short].astype(str).isin(SHORTNAMES_TO_KEEP)]

    for _, r in sub.iterrows():

        raw_short = str(r[col_short])
        hemi_raw  = str(r[col_hemi]).strip().lower()

        # normalize hemisphere
        if hemi_raw in ("left", "l"):  hemi = "lh"
        elif hemi_raw in ("right", "r"): hemi = "rh"
        elif hemi_raw in ("lh", "rh"): hemi = hemi_raw
        else:
            continue

        key = slugify(raw_short)
        key = RENAMES.get(key, key)

        # --- FIX: REMOVE SESSION SUFFIX ---
        col_name = f"{key}_{hemi}"

        # numeric convert
        val = pd.to_numeric(r[col_val], errors="coerce")

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)


# build final dataframe
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)
df_out = pd.DataFrame(rows_by_subject.values())

# ensure missing columns are filled
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan

df_out = df_out[all_columns]

# save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(10))


Saved: data\stats_t1_fs\ses1\ses1_aparc_summary_data.csv


,subject_code,meanthickness_lh,meanthickness_rh,whitesurfarea_lh,whitesurfarea_rh
0,CT002,2.68254,2.64069,89982.5,89436.7
1,NT002,2.57344,2.51046,83008.8,84464.0
2,CT003,2.62987,2.60471,87464.1,86826.2
3,NT003,2.58301,2.56652,95277.5,95072.8
4,CT004,2.59215,2.57039,95042.3,96119.6
5,CT005,2.58495,2.57033,73468.8,74068.0
6,NT005,2.64838,2.59901,77063.6,76244.8
7,NT006,2.55593,2.53523,97442.2,95931.2
8,CT007,2.46965,2.46323,85666.5,84996.4
9,NT007,2.53044,2.50443,100684.0,100008.0


## extracting from the subject file the subcortical volumes and merge nto one csv file


In [21]:
# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")
OUT_CSV = Path("data/stats_t1_fs/ses1/ses1_aseg_summary_data.csv")
SHEET_NAME = "aseg_summary"

SHORTNAME_COLUMN_CANDIDATES = ["ShortName"]
VALUE_COLUMN_CANDIDATES     = ["Value"]

SHORTNAMES_TO_KEEP = [
    "eTIV", "BrainSegVol", "lhCortexVol", "rhCortexVol",
    "CortexVol", "CerebralWhiteMatterVol", "SubCortGrayVol", "TotalGrayVol"
]

RENAMES = {}
# ----------------------------

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, recursive=False)
for path in files:

    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    # --- ONLY SESSION 1 ---
    if session != 1:
        continue

    subject_code = f"{group}{subject}"

    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    col_short = pick_column(df, SHORTNAME_COLUMN_CANDIDATES)
    col_val   = pick_column(df, VALUE_COLUMN_CANDIDATES)

    if not all([col_short, col_val]):
        print(f"Warning: missing needed columns in {path.name}. ShortName={col_short}, Value={col_val}")
        continue

    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # filter important metrics
    sub = df[df[col_short].astype(str).isin(SHORTNAMES_TO_KEEP)]

    for _, r in sub.iterrows():
        raw_short = str(r[col_short])
        key = slugify(raw_short)
        key = RENAMES.get(key, key)

        # --- FIX: REMOVE SESSION SUFFIX ---
        col_name = key

        # numeric convert
        val = pd.to_numeric(r[col_val], errors="coerce")

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

    # optional logging

# finalize dataframe
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)
df_out = pd.DataFrame(rows_by_subject.values())

# ensure all columns exist
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan

df_out = df_out[all_columns]

# save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(10))


Saved: data\stats_t1_fs\ses1\ses1_aseg_summary_data.csv


,subject_code,brainsegvol,cerebralwhitemattervol,cortexvol,etiv,lhcortexvol,rhcortexvol,subcortgrayvol,totalgrayvol
0,CT002,1206545.0,453710.0,544619.107970,1.563734e+06,274365.964887,270253.143083,54433.0,706866.107970
1,NT002,1112235.0,433453.0,479625.670030,1.410701e+06,240458.346845,239167.323185,55377.0,642291.670030
2,CT003,1175880.0,464621.0,516867.684733,1.500579e+06,260149.033135,256718.651598,56634.0,673158.684733
3,NT003,1279727.0,503653.0,547404.543520,1.657961e+06,274821.618574,272582.924946,63662.0,727368.543520
4,CT004,1238372.0,474822.0,554975.392120,1.564331e+06,275988.895743,278986.496377,59391.0,711508.392120
5,CT005,1011822.0,381493.0,434623.354034,1.320171e+06,217051.120627,217572.233407,57258.0,592673.354034
6,NT005,1094167.0,421522.0,458670.156485,1.440326e+06,231985.177486,226684.979000,54767.0,628372.156485
7,NT006,1226479.0,465770.0,552172.764328,1.567791e+06,279222.777460,272949.986868,63480.0,720886.764328
8,CT007,1135690.0,440088.0,471758.358213,1.445751e+06,236258.718813,235499.639399,60414.0,654590.358213
9,NT007,1273058.0,494327.0,573972.024005,1.585406e+06,289220.288274,284751.735731,63379.0,741652.024005


## creating csv of the merged( left and right hemishpere ) for the subcortical volumes

In [22]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
IN_CSV  = Path("data/stats_t1_fs/ses1/ses1_aparc_voxels_lh_rh.csv")
OUT_CSV = Path("data/stats_t1_fs/ses1/ses1_aparc_voxels_merge.csv")
AGGREGATION = "sum"     # "sum" or "mean"
# ----------------------------

def merge_two_cols(df, cols, how="sum"):
    """Utility to merge two numeric columns by sum or mean."""
    vals = []
    for c in cols:
        if c is not None and c in df.columns:
            vals.append(df[c])
    if not vals:
        return np.nan
    vals = pd.concat(vals, axis=1)
    return vals.mean(axis=1) if how == "mean" else vals.sum(axis=1)

# Load LH/RH file
df = pd.read_csv(IN_CSV)

# Pattern for columns like: insula_lh , insula_rh
pat = re.compile(r"^(?P<struct>.+)_(?P<hemi>lh|rh)$", re.IGNORECASE)

pairs = {}
other_cols = []

for col in df.columns:
    m = pat.match(col)
    if m:
        struct = m.group("struct")
        hemi   = m.group("hemi").lower()

        # No sessions in file → no filtering needed
        key = struct
        pairs.setdefault(key, {"lh": None, "rh": None})
        pairs[key][hemi] = col

    else:
        other_cols.append(col)

# Start output with non-hemisphere columns first (subject_code)
out = df[other_cols].copy()

# Create merged columns: "<struct>"
for struct, hemis in pairs.items():
    new_col = struct
    out[new_col] = merge_two_cols(df, [hemis.get("lh"), hemis.get("rh")], how=AGGREGATION)

# Sort columns (subject_code first)
def col_sort_key(c):
    return " 0" if c == "subject_code" else c.lower()

out = out[sorted(out.columns, key=col_sort_key)]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved merged file: {OUT_CSV}")
display(out.head(10))


Saved merged file: data\stats_t1_fs\ses1\ses1_aparc_voxels_merge.csv


,subject_code,bankssts,caudalanteriorcingulate,caudalmiddlefrontal,cuneus,entorhinal,frontalpole,fusiform,inferiorparietal,inferiortemporal,...,precentral,precuneus,rostralanteriorcingulate,rostralmiddlefrontal,superiorfrontal,superiorparietal,superiortemporal,supramarginal,temporalpole,transversetemporal
0,CT002,5637,3825,13710,6017,2730,2818,22488,35471,26203,...,30719,24249,5349,34116,53252,33087,29339,25533,3271,2266
1,NT002,4584,3611,12586,8302,3896,2921,19363,28614,24188,...,26883,19560,4607,31357,42855,28123,24566,20542,4529,1752
2,CT003,5333,3566,13370,6399,3973,2673,20230,36313,25821,...,29777,22560,4935,33854,44590,30986,24530,19014,6273,1818
3,NT003,5455,4978,15039,6948,4240,2691,20866,31815,24226,...,30344,26188,5501,38077,49655,30556,28118,26675,5240,2172
4,CT004,4984,5436,15488,6134,3623,2595,20540,35087,26348,...,31079,21325,6294,38638,55203,29889,28524,22418,4837,2528
5,CT005,3651,3113,9820,6503,3750,2656,18145,22027,20039,...,25586,16045,4068,25270,41841,23599,24882,21012,5023,1977
6,NT005,4840,3211,9124,5558,3391,2322,17926,25452,22126,...,26460,18048,3946,30740,42690,27225,23640,20317,4037,2160
7,NT006,4702,4569,15870,8678,3492,2603,22289,28662,24447,...,31148,20781,4981,34327,51212,28914,30115,26750,4731,3093
8,CT007,4131,4518,10968,5505,4377,2553,18457,25727,20893,...,27238,18471,4792,31962,45018,24517,27573,18011,4315,2102
9,NT007,5453,3833,15456,8161,3569,2625,21120,33003,24306,...,29901,25245,4757,34139,55918,35641,27164,30964,5698,2409


## creating csv of the lobes split into left and right by summign the parcels based on previous mapping

In [23]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
IN_CSV   = Path("data/stats_t1_fs/ses1/ses1_aparc_voxels_lh_rh.csv")
MAP_XLSX = Path("data/stats_t1_fs/mapping_lobes_networks.xlsx")
MAP_SHEET = "aparc_voxels_normalized_ttests"

OUT_CSV = Path("data/stats_t1_fs/ses1/ses1_aparc_lobes_lr.csv")

AGGREGATION = "sum"   # "sum" or "mean"
STRUCTNAME_COL_CANDIDATES = ["Parameter"]
LOBEFILTER_COL_CANDIDATES = ["Lobes"]
# --------------------------------

# Pattern for columns like: insula_lh or precuneus_rh
pat = re.compile(r"^(?P<struct>.+)_(?P<hemi>lh|rh)$", re.IGNORECASE)

# Load LH/RH-style file
df = pd.read_csv(IN_CSV)

# All region-level columns
lr_cols = [c for c in df.columns if pat.match(c)]

# --- 2) Load mapping for struct → lobe ---
map_df = pd.read_excel(MAP_XLSX, sheet_name=MAP_SHEET)
col_struct = pick_column(map_df, STRUCTNAME_COL_CANDIDATES)
col_lobe   = pick_column(map_df, LOBEFILTER_COL_CANDIDATES)

map_df = map_df[[col_struct, col_lobe]].dropna()
map_df[col_struct] = map_df[col_struct].astype(str)
map_df[col_lobe]   = map_df[col_lobe].astype(str)

# Mapping uses non-hemisphere names → slugify
struct_to_lobe = {
    slugify(s): slugify(l)
    for s, l in zip(map_df[col_struct], map_df[col_lobe])
}

# --- 3) Determine structs that exist in CSV ---
present_structs = {}
for c in lr_cols:
    m = pat.match(c)
    struct = slugify(m.group("struct"))
    hemi   = m.group("hemi").lower()
    present_structs.setdefault(struct, set()).add(hemi)

# Output (start with subject_code)
out = df[["subject_code"]].copy()

# Lobes present in BOTH data & mapping
lobes_present = sorted({
    struct_to_lobe.get(struct)
    for struct in present_structs
    if struct_to_lobe.get(struct) is not None
})

# --- 4) Aggregate by lobe and hemisphere ---
def reduce_cols(df, cols, how="sum"):
    if not cols:
        return np.nan
    vals = df[cols]
    return vals.mean(axis=1) if how == "mean" else vals.sum(axis=1)

for lobe in lobes_present:
    for hemi in ("lh", "rh"):
        cols_for_group = []
        for struct in present_structs:

            # skip if struct not assigned to this lobe
            if struct_to_lobe.get(struct) != lobe:
                continue

            col_name = f"{struct}_{hemi}"
            if col_name in df.columns:
                cols_for_group.append(col_name)

        # ALWAYS create a column (even if empty) for consistency
        new_col = f"{lobe}_{hemi}"
        out[new_col] = reduce_cols(df, cols_for_group, how=AGGREGATION)

# Sort columns
def order_key(c):
    return " 0" if c == "subject_code" else c.lower()

out = out[sorted(out.columns, key=order_key)]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved lobe-level file: {OUT_CSV}")
display(out.head(10))


Saved lobe-level file: data\stats_t1_fs\ses1\ses1_aparc_lobes_lr.csv


,subject_code,cingulate_lh,cingulate_rh,frontal_lh,frontal_rh,insula_lh,insula_rh,occipital_lh,occipital_rh,parietal_lh,parietal_rh,temporal_lh,temporal_rh
0,CT002,10321,9356,102820,97325,7473,7147,21689,24583,67772,70382,63946,61365
1,NT002,9761,10052,87508,82084,5971,6425,25620,27894,57266,57397,53990,54959
2,CT003,10279,9813,91427,89735,6527,6714,25541,26307,65559,67570,60496,56356
3,NT003,12000,12754,101811,97457,7572,7938,25889,26498,66390,67970,60569,59749
4,CT004,12690,12433,104498,104087,8281,7850,26906,26111,63024,65886,60145,62151
5,CT005,7822,9750,78605,77809,6550,6330,23938,22928,48274,49851,51512,50577
6,NT005,7347,8684,87199,81086,7531,7202,22162,22592,53203,56096,54254,50777
7,NT006,12661,10216,101151,98564,8291,7912,28572,31233,61578,63074,66242,61658
8,CT007,10137,10974,90324,85976,7018,7389,23967,24637,49565,53572,55316,52879
9,NT007,11191,10373,104291,98888,7791,7660,30179,31893,74785,76161,60714,59602


## creating csv of the lobes ( one for the while brain) by summing the parcels based on previous mapping


In [24]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
IN_CSV   = Path("data/stats_t1_fs/ses1/ses1_aparc_voxels_merge.csv")
MAP_XLSX = Path("data/stats_t1_fs/mapping_lobes_networks.xlsx")
MAP_SHEET = "aparc_voxels_normalized_ttests"

OUT_CSV = Path("data/stats_t1_fs/ses1/ses1_aparc_lobes_full_brain.csv")

AGGREGATION = "sum"   # "sum" or "mean"
STRUCTNAME_COL_CANDIDATES = ["Parameter"]
LOBEFILTER_COL_CANDIDATES = ["Lobes"]
# --------------------------------

# No session pattern needed – column names are just <struct>
df = pd.read_csv(IN_CSV)

# All structural columns (exclude subject_code)
struct_cols = [c for c in df.columns if c != "subject_code"]

# --- Load mapping file ---
map_df = pd.read_excel(MAP_XLSX, sheet_name=MAP_SHEET)

col_struct = pick_column(map_df, STRUCTNAME_COL_CANDIDATES)
col_lobe   = pick_column(map_df, LOBEFILTER_COL_CANDIDATES)

map_df = map_df[[col_struct, col_lobe]].dropna()
map_df[col_struct] = map_df[col_struct].astype(str)
map_df[col_lobe]   = map_df[col_lobe].astype(str)

# slugify mapping
struct_to_lobe = {
    slugify(s): slugify(l)
    for s, l in zip(map_df[col_struct], map_df[col_lobe])
}

# Determine structs present in CSV
present_structs = {slugify(c) for c in struct_cols}

# Lobes present
lobes_present = sorted({
    struct_to_lobe.get(s)
    for s in present_structs
    if struct_to_lobe.get(s) is not None
})

# Output dataframe
out = df[["subject_code"]].copy()

# Reduce per-lobe
def reduce_cols(df, cols, how="sum"):
    if not cols:
        return np.nan
    vals = df[cols]
    return vals.mean(axis=1) if how == "mean" else vals.sum(axis=1)

for lobe in lobes_present:

    cols_for_group = []

    for struct_col in struct_cols:
        if struct_to_lobe.get(slugify(struct_col)) == lobe:
            cols_for_group.append(struct_col)

    out[lobe] = reduce_cols(df, cols_for_group, how=AGGREGATION)

# Order columns
def order_key(c):
    return " 0" if c == "subject_code" else c

out = out[sorted(out.columns, key=order_key)]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved lobe-level file: {OUT_CSV}")
display(out.head(10))


Saved lobe-level file: data\stats_t1_fs\ses1\ses1_aparc_lobes_full_brain.csv


,subject_code,cingulate,frontal,insula,occipital,parietal,temporal
0,CT002,19677,200145,14620,46272,138154,125311
1,NT002,19813,169592,12396,53514,114663,108949
2,CT003,20092,181162,13241,51848,133129,116852
3,NT003,24754,199268,15510,52387,134360,120318
4,CT004,25123,208585,16131,53017,128910,122296
5,CT005,17572,156414,12880,46866,98125,102089
6,NT005,16031,168285,14733,44754,109299,105031
7,NT006,22877,199715,16203,59805,124652,127900
8,CT007,21111,176300,14407,48604,103137,108195
9,NT007,21564,203179,15451,62072,150946,120316


## creating csv of the  networks ( one for the while brain) by summing the parcels based on previous mapping


In [25]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
IN_CSV   = Path("data/stats_t1_fs/ses1/ses1_aparc_voxels_merge.csv")
MAP_XLSX = Path("data/stats_t1_fs/mapping_lobes_networks.xlsx")
MAP_SHEET = "aparc_voxels_normalized_ttests"

OUT_CSV = Path("data/stats_t1_fs/ses1/ses1_aparc_nets_full_brain.csv")

AGGREGATION = "sum"   # "sum" (default) or "mean"
STRUCTNAME_COL_CANDIDATES = ["Parameter"]
YEOFILTER_COL_CANDIDATES  = ["Yeo"]
# --------------------------------

# Pattern for columns WITHOUT sessions: <struct>
pat = re.compile(r"^(?P<struct>.+)$")

# 1) Load input table
df = pd.read_csv(IN_CSV)

# keep only structural columns (exclude subject_code)
struct_cols = [c for c in df.columns if c != "subject_code"]
df_struct = df[["subject_code"] + struct_cols].copy()

# 2) Load mapping file
map_df = pd.read_excel(MAP_XLSX, sheet_name=MAP_SHEET)

col_struct = pick_column(map_df, STRUCTNAME_COL_CANDIDATES)
col_net    = pick_column(map_df, YEOFILTER_COL_CANDIDATES)

map_df = map_df[[col_struct, col_net]].dropna()
map_df[col_struct] = map_df[col_struct].astype(str)
map_df[col_net]    = map_df[col_net].astype(str)

# slugify mapping
struct_to_net = {
    slugify(s): slugify(n)
    for s, n in zip(map_df[col_struct], map_df[col_net])
}

# Which structs exist (slugified)
present_structs = {slugify(c) for c in struct_cols}

# Networks that actually appear
networks_present = sorted({
    struct_to_net.get(s)
    for s in present_structs
    if struct_to_net.get(s)
})

# 3) Aggregate networks
out = df_struct[["subject_code"]].copy()

def reduce_cols(df, cols, how="sum"):
    if not cols:
        return np.nan
    vals = df[cols]
    return vals.mean(axis=1) if how == "mean" else vals.sum(axis=1)

for net in networks_present:
    cols_for_group = []

    for struct_col in struct_cols:
        if struct_to_net.get(slugify(struct_col)) == net:
            cols_for_group.append(struct_col)

    out[net] = reduce_cols(df_struct, cols_for_group, how=AGGREGATION)

# 4) Sort
def order_key(c):
    return " 0" if c == "subject_code" else c

out = out[sorted(out.columns, key=order_key)]

# 5) Rename networks
net_map = {
    "default_mode"      : "DMN_GM",
    "dorsal_attention"  : "DAN_GM",
    "frontoparietal"    : "FPN_GM",
    "limbic"            : "LIM_GM",
    "somatomotor"       : "SMN_GM",
    "ventral_attention" : "VAN_GM",
    "visual"            : "VIS_GM",
}

out = out.rename(columns=net_map)

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved network-level file (no sessions): {OUT_CSV}")
display(out.head(10))


Saved network-level file (no sessions): data\stats_t1_fs\ses1\ses1_aparc_nets_full_brain.csv


,subject_code,DMN_GM,DAN_GM,FPN_GM,LIM_GM,SMN_GM,VAN_GM,VIS_GM
0,CT002,174423,33087,47826,65568,91028,63487,68760
1,NT002,143547,28123,43943,60053,78053,52331,72877
2,CT003,159872,30986,47224,66238,88987,50939,72078
3,NT003,169448,30556,53116,65850,88287,66087,73253
4,CT004,172948,29889,54126,67313,90927,65302,73557
5,CT005,127745,23599,35090,55938,74339,52224,65011
6,NT005,136769,27225,39864,58400,77743,55452,62680
7,NT006,165192,28914,50197,65109,92349,67297,82094
8,CT007,142435,24517,42930,59068,79272,56471,67061
9,NT007,173935,35641,49595,66674,95932,68559,83192


## creating the final merged file

In [26]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ========== CONFIG ==========
OUTPUT_XLSX = Path("data/stats_t1_fs/ses1/ses1_T1_data.xlsx")

# The sheet that merges TWO CSVs into one
MERGED_SHEET_NAME = "global_parameters"
MERGED_CSVS = [
    Path("data/stats_t1_fs/ses1/ses1_aparc_summary_data.csv"),
    Path("data/stats_t1_fs/ses1/ses1_aseg_summary_data.csv"),
]
MERGE_KEY = "subject_code"
MERGE_HOW = "outer"           # "inner", "left", "right", "outer"
MERGE_SUFFIXES = ("_x", "_y") # for overlapping column names

# Other individual sheets: sheet_name -> csv path
OTHER_SHEETS = {
    "subcortical_vol": Path("data/stats_t1_fs/ses1/ses1_aseg_data.csv"),
    "cortical_vol_lr": Path("data/stats_t1_fs/ses1/ses1_aparc_voxels_lh_rh.csv"),
    "cortical_vol_full_brain": Path("data/stats_t1_fs/ses1/ses1_aparc_voxels_merge.csv"),
    "cortical_lobes_lr_": Path("data/stats_t1_fs/ses1/ses1_aparc_lobes_lr.csv"),
    "cortical_lobes_full_brain": Path("data/stats_t1_fs/ses1/ses1_aparc_lobes_full_brain.csv"),  # <- check your path (removed .csv.csv)
    "cortical_networks_full_brain": Path("data/stats_t1_fs/ses1/ses1_aparc_nets_full_brain.csv"),
}
# ===========================

# ---------- helpers (local to this cell) ----------

# ---------------------------------------------------

# 1) Merge the two CSVs into one sheet
df_a = _read_csv(MERGED_CSVS[0], MERGE_KEY)
df_b = _read_csv(MERGED_CSVS[1], MERGE_KEY)

if df_a.empty and df_b.empty:
    merged_df = pd.DataFrame(columns=[MERGE_KEY])
else:
    if MERGE_KEY not in df_a.columns or MERGE_KEY not in df_b.columns:
        raise ValueError(f"MERGE_KEY '{MERGE_KEY}' must be present in both files.")
    merged_df = df_a.merge(df_b, on=MERGE_KEY, how=MERGE_HOW, suffixes=MERGE_SUFFIXES)

# Keep subject_code first, then sort consistently
merged_df = _order_columns_subject_first(merged_df, MERGE_KEY)
merged_df = _sort_consistent(merged_df, MERGE_KEY)

# 2) Write everything to one Excel workbook (all sheets sorted the same way)
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    # merged sheet
    merged_df.to_excel(writer, index=False, sheet_name=MERGED_SHEET_NAME[:31])
    _autosize_columns(writer, merged_df, MERGED_SHEET_NAME[:31])

    # other sheets (one CSV per sheet)
    for sheet_name, csv_path in OTHER_SHEETS.items():
        df = _read_csv(csv_path, MERGE_KEY)
        df = _order_columns_subject_first(df, MERGE_KEY)
        df = _sort_consistent(df, MERGE_KEY)
        safe_name = sheet_name[:31]  # Excel sheet name limit is 31 chars
        df.to_excel(writer, index=False, sheet_name=safe_name)
        _autosize_columns(writer, df, safe_name)

print(f"Saved Excel workbook: {OUTPUT_XLSX}")


Saved Excel workbook: data\stats_t1_fs\ses1\ses1_T1_data.xlsx


## adding dict columsn sheet for the final merged excel

In [27]:
import pandas as pd
from pathlib import Path

MAP_XLSX = Path("data/stats_t1_fs/ses1/ses1_T1_data.xlsx")

# Load workbook
xlsx = pd.ExcelFile(MAP_XLSX)

print("📄 Sheets found in workbook:\n")

for sheet in xlsx.sheet_names:
    print(f"--- Sheet: {sheet} ---")
    df = pd.read_excel(MAP_XLSX, sheet_name=sheet)
    print("Columns:", list(df.columns))
    print()


📄 Sheets found in workbook:

--- Sheet: global_parameters ---
Columns: ['subject_code', 'meanthickness_lh', 'meanthickness_rh', 'whitesurfarea_lh', 'whitesurfarea_rh', 'brainsegvol', 'cerebralwhitemattervol', 'cortexvol', 'etiv', 'lhcortexvol', 'rhcortexvol', 'subcortgrayvol', 'totalgrayvol']

--- Sheet: subcortical_vol ---
Columns: ['subject_code', '3rd_ventricle', '4th_ventricle', '5th_ventricle', 'brain_stem', 'cc_anterior', 'cc_central', 'cc_mid_anterior', 'cc_mid_posterior', 'cc_posterior', 'csf', 'left_accumbens_area', 'left_amygdala', 'left_caudate', 'left_cerebellum_cortex', 'left_cerebellum_white_matter', 'left_choroid_plexus', 'left_hippocampus', 'left_inf_lat_vent', 'left_lateral_ventricle', 'left_non_wm_hypointensities', 'left_pallidum', 'left_putamen', 'left_thalamus', 'left_ventraldc', 'left_vessel', 'left_wm_hypointensities', 'non_wm_hypointensities', 'optic_chiasm', 'right_accumbens_area', 'right_amygdala', 'right_caudate', 'right_cerebellum_cortex', 'right_cerebellum_w

In [28]:
import pandas as pd
from openpyxl import load_workbook

# ---------- CONFIG ----------
EXCEL_PATH = "data/stats_t1_fs/ses1/ses1_T1_data.xlsx"
DICT_SHEET = "dict_columns"
# ----------------------------

# Load workbook
wb = load_workbook(EXCEL_PATH)

# If dict_columns already exists → delete it (fresh rebuild)
if DICT_SHEET in wb.sheetnames:
    wb.remove(wb[DICT_SHEET])

# Create new empty sheet
ws = wb.create_sheet(DICT_SHEET)

# Load sheet names using pandas
xlsx = pd.ExcelFile(EXCEL_PATH)

current_col = 1  # Start in column A

for sheet in xlsx.sheet_names:

    # Write sheet name in row 1
    ws.cell(row=1, column=current_col, value=sheet)

    # Load the sheet to get the columns
    df = pd.read_excel(EXCEL_PATH, sheet_name=sheet)

    # Write each column name below
    for row_i, col_name in enumerate(df.columns, start=2):
        ws.cell(row=row_i, column=current_col, value=col_name)

    # Move 3 columns right → 1 for data, 2 blank columns between blocks
    current_col += 3

# Save updated workbook (same file)
wb.save(EXCEL_PATH)

print(f"Sheet '{DICT_SHEET}' created and saved into {EXCEL_PATH}")


Sheet 'dict_columns' created and saved into data/stats_t1_fs/ses1/ses1_T1_data.xlsx


# second session

## extracting from the subject file the cortical volumes and merge nto one csv file


In [29]:

# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")  # change to your folder
OUT_CSV = Path("data/stats_t1_fs/ses2/ses2_aparc_voxels_lh_rh.csv")  # output file
SHEET_NAME = "aparc_voxels"  # sheet to read
RECURSIVE = False            # set True if files are in subfolders
# ----------------------------

file_regex = re.compile(
    r"sub-(?P<subject>\d{3})_(?P<group>CT|NT)_ses-(?P<session>[12])_merged\.xlsx$",
    re.IGNORECASE,
)

# Accumulate rows keyed by subject_code
rows_by_subject = {}
all_feature_cols = set()
files = list_excel_files(FOLDER, RECURSIVE)
for path in files:
    print("FULL PATH :", repr(str(path)))
    print("NAME ONLY :", repr(path.name))
    m = file_regex.match(path.name)
    print(m)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))
    print(session)

    # --- ONLY PROCESS SESSION 1 ---
    if session != 2:
        continue

    subject_code = f"{group}{subject}"

    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    # Locate columns
    col_struct = pick_column(df, ["StructName"])
    col_hemi   = pick_column(df, ["Hemisphere"])
    col_gray   = pick_column(df, ["GrayVol"])

    if not all([col_struct, col_hemi, col_gray]):
        print(f"Warning: missing needed columns in {path.name}. "
              f"StructName={col_struct}, Hemisphere={col_hemi}, GrayVol={col_gray}")
        continue

    # Initialize row
    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # Process rows
    for _, row in df.iterrows():
        struct = slugify(row[col_struct])
        hemi = slugify(row[col_hemi])

        if hemi in ("left", "l"): hemi = "lh"
        if hemi in ("right", "r"): hemi = "rh"

        if not struct or hemi not in ("lh", "rh"):
            continue

        # --- NEW: COLUMN NAME WITHOUT SESSION ---
        col_name = f"{struct}_{hemi}"

        # Convert to numeric
        val = row[col_gray]
        try:
            val = pd.to_numeric(val)
        except:
            val = np.nan

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)

# Build DataFrame
df_out = pd.DataFrame(rows_by_subject.values())
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan
df_out = df_out[all_columns]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(5))



FULL PATH : 'data\\stats_t1_fs\\mapping_lobes_networks.xlsx'
NAME ONLY : 'mapping_lobes_networks.xlsx'
None
FULL PATH : 'data\\stats_t1_fs\\sub-002_CT_ses-1_merged.xlsx'
NAME ONLY : 'sub-002_CT_ses-1_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-002_CT_ses-1_merged.xlsx'>
1
FULL PATH : 'data\\stats_t1_fs\\sub-002_NT_ses-1_merged.xlsx'
NAME ONLY : 'sub-002_NT_ses-1_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-002_NT_ses-1_merged.xlsx'>
1
FULL PATH : 'data\\stats_t1_fs\\sub-002_NT_ses-2_merged.xlsx'
NAME ONLY : 'sub-002_NT_ses-2_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-002_NT_ses-2_merged.xlsx'>
2
FULL PATH : 'data\\stats_t1_fs\\sub-003_CT_ses-1_merged.xlsx'
NAME ONLY : 'sub-003_CT_ses-1_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-003_CT_ses-1_merged.xlsx'>
1
FULL PATH : 'data\\stats_t1_fs\\sub-003_CT_ses-2_merged.xlsx'
NAME ONLY : 'sub-003_CT_ses-2_merged.xlsx'
<re.Match object; span=(0, 28), match='sub-003_CT_ses-2_merged.xlsx'>
2
FULL PA

,subject_code,bankssts_lh,bankssts_rh,caudalanteriorcingulate_lh,caudalanteriorcingulate_rh,caudalmiddlefrontal_lh,caudalmiddlefrontal_rh,cuneus_lh,cuneus_rh,entorhinal_lh,...,superiorparietal_lh,superiorparietal_rh,superiortemporal_lh,superiortemporal_rh,supramarginal_lh,supramarginal_rh,temporalpole_lh,temporalpole_rh,transversetemporal_lh,transversetemporal_rh
0,NT002,2644,1935,1446,1999,5706,5937,3152,5006,1523,...,13392,14360,11952,12036,10211,9902,2388,2386,961,816
1,CT003,2449,2528,1693,1978,7022,5961,2956,3309,1987,...,14936,14817,11607,12440,9317,9477,3147,2882,950,757
2,NT005,2611,2153,975,2121,3774,4359,2451,2767,2010,...,11910,12556,11662,10863,9166,9764,2690,2699,1117,805
3,CT007,2020,2106,1813,2729,5464,6269,2844,2909,1888,...,12173,12493,13761,12980,9319,8923,3149,2569,1195,750
4,CT010,2480,2242,1481,2313,5247,3316,3048,3672,1536,...,14381,9522,12082,10763,13357,6827,2289,2235,1127,800


## extracting from the subject file subcortical   volumes and merge nto one csv file


In [30]:
# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")
OUT_CSV = Path("data/stats_t1_fs/ses2/ses2_aseg_data.csv")
SHEET_NAME = "aseg_voxels"
RECURSIVE = False
# ----------------------------

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, RECURSIVE)
for path in files:

    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    # --- ONLY USE SESSION 1 ---
    if session != 2:
        continue

    subject_code = f"{group}{subject}"

    # read the aseg sheet
    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    # locate required columns
    col_struct = pick_column(df, ["StructName"])
    col_gray   = pick_column(df, ["Volume_mm3"])

    if not all([col_struct, col_gray]):
        print(f"Warning: missing needed columns in {path.name}. struct={col_struct}, vol={col_gray}")
        continue

    # initialize subject row
    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # iterate rows
    for _, row in df.iterrows():
        struct = slugify(row[col_struct])
        if not struct:
            continue

        # --- FIX: NO SESSION SUFFIX ---
        col_name = struct

        val = row[col_gray]
        try:
            val = pd.to_numeric(val)
        except:
            val = np.nan

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

    # optional logging


# final table columns
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)

# build wide dataframe
df_out = pd.DataFrame(rows_by_subject.values())

# ensure missing columns exist
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan

df_out = df_out[all_columns]

# write CSV
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(5))


Saved: data\stats_t1_fs\ses2\ses2_aseg_data.csv


,subject_code,3rd_ventricle,4th_ventricle,5th_ventricle,brain_stem,cc_anterior,cc_central,cc_mid_anterior,cc_mid_posterior,cc_posterior,...,right_inf_lat_vent,right_lateral_ventricle,right_non_wm_hypointensities,right_pallidum,right_putamen,right_thalamus,right_ventraldc,right_vessel,right_wm_hypointensities,wm_hypointensities
0,NT002,848.4,905.7,0.0,20227.2,548.6,628.3,438.2,413.8,924.2,...,256.2,3414.7,0.0,1773.5,4982.9,6935.9,4155.3,27.5,0.0,534.4
1,CT003,862.2,1549.3,0.0,20067.4,831.1,802.5,562.7,609.9,1018.2,...,375.8,5374.1,0.0,2034.5,4818.3,7209.8,3917.4,9.2,0.0,809.0
2,NT005,797.4,1390.6,0.0,20013.8,1070.8,558.3,543.2,550.4,980.0,...,206.8,5780.1,0.0,1825.2,4853.3,6618.6,3493.4,41.5,0.0,340.5
3,CT007,828.3,2001.5,0.0,23184.8,1042.7,555.1,495.4,513.4,903.0,...,493.2,3372.9,0.0,2095.8,4702.1,8321.4,4114.4,7.6,0.0,685.5
4,CT010,929.1,2334.6,0.0,22725.4,949.6,569.6,579.7,633.7,1000.1,...,181.7,6904.5,0.0,1988.0,4664.7,7374.2,4148.9,10.6,0.0,535.8


## extracting from the subject file white matter  volumes and merge nto one csv file


In [31]:
# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")
OUT_CSV = Path("data/stats_t1_fs/ses2/ses2_wmparc_data.csv")
SHEET_NAME = "wmparc_voxels"
RECURSIVE = False
# ----------------------------

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, RECURSIVE)
for path in files:

    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    # --- ONLY USE SESSION 2 ---
    if session != 2:
        continue

    subject_code = f"{group}{subject}"

    # Load wmparc sheet
    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    # Identify needed columns
    col_struct = pick_column(df, ["StructName"])
    col_gray   = pick_column(df, ["Volume_mm3"])

    if not all([col_struct, col_gray]):
        print(f"Warning: missing needed columns in {path.name}. struct={col_struct}, vol={col_gray}")
        continue

    # Initialize subject row
    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # Process each ROI row
    for _, row in df.iterrows():
        struct = slugify(row[col_struct])
        if not struct:
            continue

        # --- FIX: remove session suffix ---
        col_name = struct

        # convert to numeric
        val = row[col_gray]
        try:
            val = pd.to_numeric(val)
        except:
            val = np.nan

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

    # optional tracking

# Final set of columns
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)

# Build final table
df_out = pd.DataFrame(rows_by_subject.values())

# Ensure all expected columns exist
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan

df_out = df_out[all_columns]

# Save CSV
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(5))


Saved: data\stats_t1_fs\ses2\ses2_wmparc_data.csv


,subject_code,left_unsegmentedwhitematter,right_unsegmentedwhitematter,wm_lh_bankssts,wm_lh_caudalanteriorcingulate,wm_lh_caudalmiddlefrontal,wm_lh_cuneus,wm_lh_entorhinal,wm_lh_frontalpole,wm_lh_fusiform,...,wm_rh_precentral,wm_rh_precuneus,wm_rh_rostralanteriorcingulate,wm_rh_rostralmiddlefrontal,wm_rh_superiorfrontal,wm_rh_superiorparietal,wm_rh_superiortemporal,wm_rh_supramarginal,wm_rh_temporalpole,wm_rh_transversetemporal
0,NT002,28138.1,27934.5,3346.1,2363.5,5982.7,3255.9,935.9,345.5,5776.8,...,11894.0,7826.5,1654.8,11424.0,14914.6,11592.3,6787.5,7414.4,658.3,478.0
1,CT003,34201.5,32796.0,3869.8,3267.3,6825.3,2413.0,1057.0,251.3,7752.2,...,14339.5,10000.5,1983.3,12883.4,16902.2,12273.0,6022.7,8070.5,832.0,504.7
2,NT005,29960.9,29078.1,2449.1,1753.1,5141.3,2153.2,1091.9,316.1,6663.4,...,11004.1,8994.9,1718.3,11836.7,15599.8,11877.5,6080.8,8108.6,644.7,422.8
3,CT007,27850.0,26192.0,2645.3,2805.1,5603.7,2895.4,983.7,294.7,6849.4,...,14219.8,9203.9,2052.4,11762.2,17044.1,10797.5,6750.9,7203.6,774.1,418.0
4,CT010,30400.2,29906.1,2644.2,2442.0,5439.7,2033.0,834.0,245.2,5858.6,...,8442.9,6909.4,1882.0,18911.9,18284.5,9114.2,5758.8,4446.7,692.8,615.9


## extracting the cortical parmeters of each subject into one csv

In [32]:
# Build per-subject table from aparc_summary, including hemisphere in column names.
# Requires helpers already defined: file_regex, list_excel_files, slugify, pick_column, col_sort_key, mark_extraction_complete

from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")
OUT_CSV = Path("data/stats_t1_fs/ses2/ses2_aparc_summary_data.csv")
SHEET_NAME = "aparc_summary"

SHORTNAME_COLUMN_CANDIDATES = ["ShortName"]
VALUE_COLUMN_CANDIDATES     = ["Value"]
HEMI_COLUMN_CANDIDATES      = ["Hemisphere"]

SHORTNAMES_TO_KEEP = ["WhiteSurfArea", "MeanThickness"]

RENAMES = {}
# ----------------------------

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, recursive=False)
for path in files:

    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    # --- ONLY SESSION 2---
    if session != 2:
        continue

    subject_code = f"{group}{subject}"

    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    col_short = pick_column(df, SHORTNAME_COLUMN_CANDIDATES)
    col_val   = pick_column(df, VALUE_COLUMN_CANDIDATES)
    col_hemi  = pick_column(df, HEMI_COLUMN_CANDIDATES)

    if not all([col_short, col_val, col_hemi]):
        print(f"Warning: missing needed columns in {path.name}. "
              f"ShortName={col_short}, Value={col_val}, Hemisphere={col_hemi}")
        continue

    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # keep only desired ShortNames
    sub = df[df[col_short].astype(str).isin(SHORTNAMES_TO_KEEP)]

    for _, r in sub.iterrows():

        raw_short = str(r[col_short])
        hemi_raw  = str(r[col_hemi]).strip().lower()

        # normalize hemisphere
        if hemi_raw in ("left", "l"):  hemi = "lh"
        elif hemi_raw in ("right", "r"): hemi = "rh"
        elif hemi_raw in ("lh", "rh"): hemi = hemi_raw
        else:
            continue

        key = slugify(raw_short)
        key = RENAMES.get(key, key)

        # --- FIX: REMOVE SESSION SUFFIX ---
        col_name = f"{key}_{hemi}"

        # numeric convert
        val = pd.to_numeric(r[col_val], errors="coerce")

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)


# build final dataframe
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)
df_out = pd.DataFrame(rows_by_subject.values())

# ensure missing columns are filled
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan

df_out = df_out[all_columns]

# save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(10))


Saved: data\stats_t1_fs\ses2\ses2_aparc_summary_data.csv


,subject_code,meanthickness_lh,meanthickness_rh,whitesurfarea_lh,whitesurfarea_rh
0,NT002,2.53983,2.47017,82790.9,83587.8
1,CT003,2.61352,2.60654,86437.2,85357.4
2,NT005,2.60428,2.57512,74623.6,74142.1
3,CT007,2.51822,2.47898,84689.8,83382.3
4,CT010,2.54736,2.53174,80297.0,79914.2
5,CT012,2.62747,2.66530,80167.3,79735.6
6,CT013,2.57238,2.52014,82662.4,83327.8
7,NT015,2.57023,2.53286,76559.3,76265.9
8,NT016,2.51704,2.58262,81984.0,81744.8
9,NT017,2.55015,2.52395,80126.5,78178.8


## extractign from each subject the subcortical parameters and merge into one csv

In [33]:
# ---------- CONFIG ----------
FOLDER = Path("data/stats_t1_fs")
OUT_CSV = Path("data/stats_t1_fs/ses2/ses2_aseg_summary_data.csv")
SHEET_NAME = "aseg_summary"

SHORTNAME_COLUMN_CANDIDATES = ["ShortName"]
VALUE_COLUMN_CANDIDATES     = ["Value"]

SHORTNAMES_TO_KEEP = [
    "eTIV", "BrainSegVol", "lhCortexVol", "rhCortexVol",
    "CortexVol", "CerebralWhiteMatterVol", "SubCortGrayVol", "TotalGrayVol"
]

RENAMES = {}
# ----------------------------

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, recursive=False)
for path in files:

    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    # --- ONLY SESSION 2 ---
    if session != 2:
        continue

    subject_code = f"{group}{subject}"

    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    col_short = pick_column(df, SHORTNAME_COLUMN_CANDIDATES)
    col_val   = pick_column(df, VALUE_COLUMN_CANDIDATES)

    if not all([col_short, col_val]):
        print(f"Warning: missing needed columns in {path.name}. ShortName={col_short}, Value={col_val}")
        continue

    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # filter important metrics
    sub = df[df[col_short].astype(str).isin(SHORTNAMES_TO_KEEP)]

    for _, r in sub.iterrows():
        raw_short = str(r[col_short])
        key = slugify(raw_short)
        key = RENAMES.get(key, key)

        # --- FIX: REMOVE SESSION SUFFIX ---
        col_name = key

        # numeric convert
        val = pd.to_numeric(r[col_val], errors="coerce")

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

    # optional logging

# finalize dataframe
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)
df_out = pd.DataFrame(rows_by_subject.values())

# ensure all columns exist
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan

df_out = df_out[all_columns]

# save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(10))


Saved: data\stats_t1_fs\ses2\ses2_aseg_summary_data.csv


,subject_code,brainsegvol,cerebralwhitemattervol,cortexvol,etiv,lhcortexvol,rhcortexvol,subcortgrayvol,totalgrayvol
0,NT002,1098726.0,432028.0,467041.830018,1.426071e+06,235970.986784,231070.843234,55652.0,630349.830018
1,CT003,1164172.0,466003.0,504776.174155,1.496023e+06,253436.665847,251339.508308,56449.0,660946.174155
2,NT005,1070379.0,419660.0,435276.918266,1.382242e+06,219169.291362,216107.626903,52020.0,602504.918266
3,CT007,1135751.0,435282.0,475457.080609,1.431293e+06,240910.228531,234546.852078,61610.0,660398.080609
4,CT010,1112571.0,422521.0,455246.833629,1.428136e+06,228720.549249,226526.284380,56680.0,636282.833629
5,CT012,1093848.0,403552.0,476809.407913,1.434673e+06,237350.015321,239459.392592,58747.0,641977.407913
6,CT013,1072627.0,391345.0,478995.662613,1.384159e+06,240920.042524,238075.620090,56453.0,644256.662613
7,NT015,1031328.0,400304.0,437612.253679,1.317540e+06,220408.214312,217204.039368,53839.0,597344.253679
8,NT016,1104085.0,407686.0,468350.108376,1.257785e+06,231597.223154,236752.885222,54622.0,654545.108376
9,NT017,1079215.0,415807.0,447924.459215,1.392282e+06,227413.110316,220511.348899,52490.0,613702.459215


## crate a csv files ov the parces in the cortex along the brain ( not split between the right and left side)

In [34]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
IN_CSV  = Path("data/stats_t1_fs/ses2/ses2_aparc_voxels_lh_rh.csv")
OUT_CSV = Path("data/stats_t1_fs/ses2/ses2_aparc_voxels_merge.csv")
AGGREGATION = "sum"     # "sum" or "mean"
# ----------------------------

def merge_two_cols(df, cols, how="sum"):
    """Utility to merge two numeric columns by sum or mean."""
    vals = []
    for c in cols:
        if c is not None and c in df.columns:
            vals.append(df[c])
    if not vals:
        return np.nan
    vals = pd.concat(vals, axis=1)
    return vals.mean(axis=1) if how == "mean" else vals.sum(axis=1)

# Load LH/RH file
df = pd.read_csv(IN_CSV)

# Pattern for columns like: insula_lh , insula_rh
pat = re.compile(r"^(?P<struct>.+)_(?P<hemi>lh|rh)$", re.IGNORECASE)

pairs = {}
other_cols = []

for col in df.columns:
    m = pat.match(col)
    if m:
        struct = m.group("struct")
        hemi   = m.group("hemi").lower()

        # No sessions in file → no filtering needed
        key = struct
        pairs.setdefault(key, {"lh": None, "rh": None})
        pairs[key][hemi] = col

    else:
        other_cols.append(col)

# Start output with non-hemisphere columns first (subject_code)
out = df[other_cols].copy()

# Create merged columns: "<struct>"
for struct, hemis in pairs.items():
    new_col = struct
    out[new_col] = merge_two_cols(df, [hemis.get("lh"), hemis.get("rh")], how=AGGREGATION)

# Sort columns (subject_code first)
def col_sort_key(c):
    return " 0" if c == "subject_code" else c.lower()

out = out[sorted(out.columns, key=col_sort_key)]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved merged file: {OUT_CSV}")
display(out.head(10))


Saved merged file: data\stats_t1_fs\ses2\ses2_aparc_voxels_merge.csv


,subject_code,bankssts,caudalanteriorcingulate,caudalmiddlefrontal,cuneus,entorhinal,frontalpole,fusiform,inferiorparietal,inferiortemporal,...,precentral,precuneus,rostralanteriorcingulate,rostralmiddlefrontal,superiorfrontal,superiorparietal,superiortemporal,supramarginal,temporalpole,transversetemporal
0,NT002,4579,3445,11643,8158,3199,3072,19352,28076,23250,...,25270,19127,4741,31531,41715,27752,23988,20113,4774,1777
1,CT003,4977,3671,12983,6265,3644,2592,20023,35161,25432,...,29073,21454,5092,32926,43905,29753,24047,18794,6029,1707
2,NT005,4764,3096,8133,5218,3618,2136,18624,24135,21707,...,23671,16700,4266,29028,38787,24466,22525,18930,5389,1922
3,CT007,4126,4542,11733,5753,4160,2510,18863,25535,20782,...,27401,18594,5008,32675,45778,24666,26741,18242,5718,1945
4,CT010,4722,3794,8563,6720,3180,2639,18998,24366,19047,...,20727,18586,4952,33094,45005,23903,22845,20184,4524,1927
5,CT012,5359,3855,10957,6092,3903,2211,22157,25096,24732,...,25455,18884,4322,31537,43770,20835,27560,20464,5831,1965
6,CT013,4962,4204,11514,6431,3344,2798,21859,28499,22234,...,23758,18775,4705,30728,44717,25206,23373,18521,3612,2052
7,NT015,4340,3074,11861,8074,3075,2035,16821,25314,21644,...,24127,18504,4218,27481,35196,22482,22399,19109,3679,1596
8,NT016,5390,4290,10152,6203,3971,2599,20322,26345,22672,...,24038,19590,4763,29267,38342,28569,24163,23431,4443,2484
9,NT017,6238,3007,10445,5343,3779,2158,20003,30572,21390,...,21366,19577,3806,27286,38201,23330,23373,21716,4386,2029


## merge between parcels along the hemispheres to create lobes and arrange the into one csv

In [35]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
IN_CSV   = Path("data/stats_t1_fs/ses2/ses2_aparc_voxels_lh_rh.csv")
MAP_XLSX = Path("data/stats_t1_fs/mapping_lobes_networks.xlsx")
MAP_SHEET = "aparc_voxels_normalized_ttests"

OUT_CSV = Path("data/stats_t1_fs/ses2/ses2_aparc_lobes_lr.csv")

AGGREGATION = "sum"   # "sum" or "mean"
STRUCTNAME_COL_CANDIDATES = ["Parameter"]
LOBEFILTER_COL_CANDIDATES = ["Lobes"]
# --------------------------------

# Pattern for columns like: insula_lh or precuneus_rh
pat = re.compile(r"^(?P<struct>.+)_(?P<hemi>lh|rh)$", re.IGNORECASE)

# Load LH/RH-style file
df = pd.read_csv(IN_CSV)

# All region-level columns
lr_cols = [c for c in df.columns if pat.match(c)]

# --- 2) Load mapping for struct → lobe ---
map_df = pd.read_excel(MAP_XLSX, sheet_name=MAP_SHEET)
col_struct = pick_column(map_df, STRUCTNAME_COL_CANDIDATES)
col_lobe   = pick_column(map_df, LOBEFILTER_COL_CANDIDATES)

map_df = map_df[[col_struct, col_lobe]].dropna()
map_df[col_struct] = map_df[col_struct].astype(str)
map_df[col_lobe]   = map_df[col_lobe].astype(str)

# Mapping uses non-hemisphere names → slugify
struct_to_lobe = {
    slugify(s): slugify(l)
    for s, l in zip(map_df[col_struct], map_df[col_lobe])
}

# --- 3) Determine structs that exist in CSV ---
present_structs = {}
for c in lr_cols:
    m = pat.match(c)
    struct = slugify(m.group("struct"))
    hemi   = m.group("hemi").lower()
    present_structs.setdefault(struct, set()).add(hemi)

# Output (start with subject_code)
out = df[["subject_code"]].copy()

# Lobes present in BOTH data & mapping
lobes_present = sorted({
    struct_to_lobe.get(struct)
    for struct in present_structs
    if struct_to_lobe.get(struct) is not None
})

# --- 4) Aggregate by lobe and hemisphere ---
def reduce_cols(df, cols, how="sum"):
    if not cols:
        return np.nan
    vals = df[cols]
    return vals.mean(axis=1) if how == "mean" else vals.sum(axis=1)

for lobe in lobes_present:
    for hemi in ("lh", "rh"):
        cols_for_group = []
        for struct in present_structs:

            # skip if struct not assigned to this lobe
            if struct_to_lobe.get(struct) != lobe:
                continue

            col_name = f"{struct}_{hemi}"
            if col_name in df.columns:
                cols_for_group.append(col_name)

        # ALWAYS create a column (even if empty) for consistency
        new_col = f"{lobe}_{hemi}"
        out[new_col] = reduce_cols(df, cols_for_group, how=AGGREGATION)

# Sort columns
def order_key(c):
    return " 0" if c == "subject_code" else c.lower()

out = out[sorted(out.columns, key=order_key)]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved lobe-level file: {OUT_CSV}")
display(out.head(10))


Saved lobe-level file: data\stats_t1_fs\ses2\ses2_aparc_lobes_lr.csv


,subject_code,cingulate_lh,cingulate_rh,frontal_lh,frontal_rh,insula_lh,insula_rh,occipital_lh,occipital_rh,parietal_lh,parietal_rh,temporal_lh,temporal_rh
0,NT002,9525,9767,84987,78929,6454,6136,25121,27506,56993,55219,52785,53510
1,CT003,10438,9949,89117,88941,6563,6085,24755,25926,63330,65360,58963,54846
2,NT005,7271,8535,79151,75220,8017,7694,21182,21997,49062,51324,54209,51011
3,CT007,9926,11358,92743,86548,7207,7242,24109,24609,50691,52915,56065,51659
4,CT010,9800,10009,79270,85702,6363,6153,25503,27415,56446,46166,51053,50793
5,CT012,9596,10471,85530,83433,7577,7480,23507,24856,50048,51916,60694,60971
6,CT013,10936,10014,87884,84834,6784,7060,27218,27059,53481,54265,54676,54525
7,NT015,9468,7617,74930,74928,6483,6059,26398,26922,51254,51356,51652,49947
8,NT016,9289,10992,79260,79476,6582,6748,22572,27181,56742,56610,56679,55316
9,NT017,9217,7526,78777,77023,6701,6647,21193,22112,55769,54753,55440,52108


## creating a file of the lobes per brain and arrange this into csv

In [36]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
IN_CSV   = Path("data/stats_t1_fs/ses2/ses2_aparc_voxels_merge.csv")
MAP_XLSX = Path("data/stats_t1_fs/mapping_lobes_networks.xlsx")
MAP_SHEET = "aparc_voxels_normalized_ttests"

OUT_CSV = Path("data/stats_t1_fs/ses2/ses2_aparc_lobes_full_brain.csv")

AGGREGATION = "sum"   # "sum" or "mean"
STRUCTNAME_COL_CANDIDATES = ["Parameter"]
LOBEFILTER_COL_CANDIDATES = ["Lobes"]
# --------------------------------

# No session pattern needed – column names are just <struct>
df = pd.read_csv(IN_CSV)

# All structural columns (exclude subject_code)
struct_cols = [c for c in df.columns if c != "subject_code"]

# --- Load mapping file ---
map_df = pd.read_excel(MAP_XLSX, sheet_name=MAP_SHEET)

col_struct = pick_column(map_df, STRUCTNAME_COL_CANDIDATES)
col_lobe   = pick_column(map_df, LOBEFILTER_COL_CANDIDATES)

map_df = map_df[[col_struct, col_lobe]].dropna()
map_df[col_struct] = map_df[col_struct].astype(str)
map_df[col_lobe]   = map_df[col_lobe].astype(str)

# slugify mapping
struct_to_lobe = {
    slugify(s): slugify(l)
    for s, l in zip(map_df[col_struct], map_df[col_lobe])
}

# Determine structs present in CSV
present_structs = {slugify(c) for c in struct_cols}

# Lobes present
lobes_present = sorted({
    struct_to_lobe.get(s)
    for s in present_structs
    if struct_to_lobe.get(s) is not None
})

# Output dataframe
out = df[["subject_code"]].copy()

# Reduce per-lobe
def reduce_cols(df, cols, how="sum"):
    if not cols:
        return np.nan
    vals = df[cols]
    return vals.mean(axis=1) if how == "mean" else vals.sum(axis=1)

for lobe in lobes_present:

    cols_for_group = []

    for struct_col in struct_cols:
        if struct_to_lobe.get(slugify(struct_col)) == lobe:
            cols_for_group.append(struct_col)

    out[lobe] = reduce_cols(df, cols_for_group, how=AGGREGATION)

# Order columns
def order_key(c):
    return " 0" if c == "subject_code" else c

out = out[sorted(out.columns, key=order_key)]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved lobe-level file: {OUT_CSV}")
display(out.head(10))


Saved lobe-level file: data\stats_t1_fs\ses2\ses2_aparc_lobes_full_brain.csv


,subject_code,cingulate,frontal,insula,occipital,parietal,temporal
0,NT002,19292,163916,12590,52627,112212,106295
1,CT003,20387,178058,12648,50681,128690,113809
2,NT005,15806,154371,15711,43179,100386,105220
3,CT007,21284,179291,14449,48718,103606,107724
4,CT010,19809,164972,12516,52918,102612,101846
5,CT012,20067,168963,15057,48363,101964,121665
6,CT013,20950,172718,13844,54277,107746,109201
7,NT015,17085,149858,12542,53320,102610,101599
8,NT016,20281,158736,13330,49753,113352,111995
9,NT017,16743,155800,13348,43305,110522,107548


## creatnf csv file for the networks for the whole brain based on previous mappping

In [37]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
IN_CSV   = Path("data/stats_t1_fs/ses2/ses2_aparc_voxels_merge.csv")
MAP_XLSX = Path("data/stats_t1_fs/mapping_lobes_networks.xlsx")
MAP_SHEET = "aparc_voxels_normalized_ttests"

OUT_CSV = Path("data/stats_t1_fs/ses2/ses2_aparc_nets_full_brain.csv")

AGGREGATION = "sum"   # "sum" (default) or "mean"
STRUCTNAME_COL_CANDIDATES = ["Parameter"]
YEOFILTER_COL_CANDIDATES  = ["Yeo"]
# --------------------------------

# Pattern for columns WITHOUT sessions: <struct>
pat = re.compile(r"^(?P<struct>.+)$")

# 1) Load input table
df = pd.read_csv(IN_CSV)

# keep only structural columns (exclude subject_code)
struct_cols = [c for c in df.columns if c != "subject_code"]
df_struct = df[["subject_code"] + struct_cols].copy()

# 2) Load mapping file
map_df = pd.read_excel(MAP_XLSX, sheet_name=MAP_SHEET)

col_struct = pick_column(map_df, STRUCTNAME_COL_CANDIDATES)
col_net    = pick_column(map_df, YEOFILTER_COL_CANDIDATES)

map_df = map_df[[col_struct, col_net]].dropna()
map_df[col_struct] = map_df[col_struct].astype(str)
map_df[col_net]    = map_df[col_net].astype(str)

# slugify mapping
struct_to_net = {
    slugify(s): slugify(n)
    for s, n in zip(map_df[col_struct], map_df[col_net])
}

# Which structs exist (slugified)
present_structs = {slugify(c) for c in struct_cols}

# Networks that actually appear
networks_present = sorted({
    struct_to_net.get(s)
    for s in present_structs
    if struct_to_net.get(s)
})

# 3) Aggregate networks
out = df_struct[["subject_code"]].copy()

def reduce_cols(df, cols, how="sum"):
    if not cols:
        return np.nan
    vals = df[cols]
    return vals.mean(axis=1) if how == "mean" else vals.sum(axis=1)

for net in networks_present:
    cols_for_group = []

    for struct_col in struct_cols:
        if struct_to_net.get(slugify(struct_col)) == net:
            cols_for_group.append(struct_col)

    out[net] = reduce_cols(df_struct, cols_for_group, how=AGGREGATION)

# 4) Sort
def order_key(c):
    return " 0" if c == "subject_code" else c

out = out[sorted(out.columns, key=order_key)]

# 5) Rename networks
net_map = {
    "default_mode"      : "DMN_GM",
    "dorsal_attention"  : "DAN_GM",
    "frontoparietal"    : "FPN_GM",
    "limbic"            : "LIM_GM",
    "somatomotor"       : "SMN_GM",
    "ventral_attention" : "VAN_GM",
    "visual"            : "VIS_GM",
}

out = out.rename(columns=net_map)

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved network-level file (no sessions): {OUT_CSV}")
display(out.head(10))


Saved network-level file (no sessions): data\stats_t1_fs\ses2\ses2_aparc_nets_full_brain.csv


,subject_code,DMN_GM,DAN_GM,FPN_GM,LIM_GM,SMN_GM,VAN_GM,VIS_GM
0,NT002,139908,27752,43174,57681,74665,51773,71979
1,CT003,155634,29753,45909,65389,86844,50040,70704
2,NT005,129439,24466,37161,57323,70700,53781,61803
3,CT007,142120,24666,44408,60856,78762,56679,67581
4,CT010,140862,23903,41657,53975,68826,53534,71916
5,CT012,145244,20835,42494,62125,78424,56437,70520
6,CT013,147309,25206,42242,59456,73384,55003,76136
7,NT015,130953,22482,39342,53820,71315,48961,70141
8,NT016,139734,28569,39419,59945,72395,57310,70075
9,NT017,139925,23330,37731,57606,68591,56775,63308


## creating the merged final file

In [38]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ========== CONFIG ==========
OUTPUT_XLSX = Path("data/stats_t1_fs/ses2/ses2_T1_data.xlsx")

# The sheet that merges TWO CSVs into one
MERGED_SHEET_NAME = "global_parameters"
MERGED_CSVS = [
    Path("data/stats_t1_fs/ses2/ses2_aparc_summary_data.csv"),
    Path("data/stats_t1_fs/ses2/ses2_aseg_summary_data.csv"),
]
MERGE_KEY = "subject_code"
MERGE_HOW = "outer"           # "inner", "left", "right", "outer"
MERGE_SUFFIXES = ("_x", "_y") # for overlapping column names

# Other individual sheets: sheet_name -> csv path
OTHER_SHEETS = {
    "subcortical_vol": Path("data/stats_t1_fs/ses2/ses2_aseg_data.csv"),
    "cortical_vol_lr": Path("data/stats_t1_fs/ses2/ses2_aparc_voxels_lh_rh.csv"),
    "cortical_vol_full_brain": Path("data/stats_t1_fs/ses2/ses2_aparc_voxels_merge.csv"),
    "cortical_lobes_lr_": Path("data/stats_t1_fs/ses2/ses2_aparc_lobes_lr.csv"),
    "cortical_lobes_full_brain": Path("data/stats_t1_fs/ses2/ses2_aparc_lobes_full_brain.csv"),  # <- check your path (removed .csv.csv)
    "cortical_networks_full_brain": Path("data/stats_t1_fs/ses2/ses2_aparc_nets_full_brain.csv"),
}
# ===========================

# ---------- helpers (local to this cell) ----------

# ---------------------------------------------------

# 1) Merge the two CSVs into one sheet
df_a = _read_csv(MERGED_CSVS[0], MERGE_KEY)
df_b = _read_csv(MERGED_CSVS[1], MERGE_KEY)

if df_a.empty and df_b.empty:
    merged_df = pd.DataFrame(columns=[MERGE_KEY])
else:
    if MERGE_KEY not in df_a.columns or MERGE_KEY not in df_b.columns:
        raise ValueError(f"MERGE_KEY '{MERGE_KEY}' must be present in both files.")
    merged_df = df_a.merge(df_b, on=MERGE_KEY, how=MERGE_HOW, suffixes=MERGE_SUFFIXES)

# Keep subject_code first, then sort consistently
merged_df = _order_columns_subject_first(merged_df, MERGE_KEY)
merged_df = _sort_consistent(merged_df, MERGE_KEY)

# 2) Write everything to one Excel workbook (all sheets sorted the same way)
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    # merged sheet
    merged_df.to_excel(writer, index=False, sheet_name=MERGED_SHEET_NAME[:31])
    _autosize_columns(writer, merged_df, MERGED_SHEET_NAME[:31])

    # other sheets (one CSV per sheet)
    for sheet_name, csv_path in OTHER_SHEETS.items():
        df = _read_csv(csv_path, MERGE_KEY)
        df = _order_columns_subject_first(df, MERGE_KEY)
        df = _sort_consistent(df, MERGE_KEY)
        safe_name = sheet_name[:31]  # Excel sheet name limit is 31 chars
        df.to_excel(writer, index=False, sheet_name=safe_name)
        _autosize_columns(writer, df, safe_name)

print(f"Saved Excel workbook: {OUTPUT_XLSX}")


Saved Excel workbook: data\stats_t1_fs\ses2\ses2_T1_data.xlsx


## adding the dict column to the column dict sheet

In [39]:
import pandas as pd
from pathlib import Path

MAP_XLSX = Path("data/stats_t1_fs/ses2/ses2_T1_data.xlsx")

# Load workbook
xlsx = pd.ExcelFile(MAP_XLSX)

print("📄 Sheets found in workbook:\n")

for sheet in xlsx.sheet_names:
    print(f"--- Sheet: {sheet} ---")
    df = pd.read_excel(MAP_XLSX, sheet_name=sheet)
    print("Columns:", list(df.columns))
    print()


📄 Sheets found in workbook:

--- Sheet: global_parameters ---
Columns: ['subject_code', 'meanthickness_lh', 'meanthickness_rh', 'whitesurfarea_lh', 'whitesurfarea_rh', 'brainsegvol', 'cerebralwhitemattervol', 'cortexvol', 'etiv', 'lhcortexvol', 'rhcortexvol', 'subcortgrayvol', 'totalgrayvol']

--- Sheet: subcortical_vol ---
Columns: ['subject_code', '3rd_ventricle', '4th_ventricle', '5th_ventricle', 'brain_stem', 'cc_anterior', 'cc_central', 'cc_mid_anterior', 'cc_mid_posterior', 'cc_posterior', 'csf', 'left_accumbens_area', 'left_amygdala', 'left_caudate', 'left_cerebellum_cortex', 'left_cerebellum_white_matter', 'left_choroid_plexus', 'left_hippocampus', 'left_inf_lat_vent', 'left_lateral_ventricle', 'left_non_wm_hypointensities', 'left_pallidum', 'left_putamen', 'left_thalamus', 'left_ventraldc', 'left_vessel', 'left_wm_hypointensities', 'non_wm_hypointensities', 'optic_chiasm', 'right_accumbens_area', 'right_amygdala', 'right_caudate', 'right_cerebellum_cortex', 'right_cerebellum_w

In [40]:
import pandas as pd
from openpyxl import load_workbook

# ---------- CONFIG ----------
EXCEL_PATH = "data/stats_t1_fs/ses2/ses2_T1_data.xlsx"
DICT_SHEET = "dict_columns"
# ----------------------------

# Load workbook
wb = load_workbook(EXCEL_PATH)

# If dict_columns already exists → delete it (fresh rebuild)
if DICT_SHEET in wb.sheetnames:
    wb.remove(wb[DICT_SHEET])

# Create new empty sheet
ws = wb.create_sheet(DICT_SHEET)

# Load sheet names using pandas
xlsx = pd.ExcelFile(EXCEL_PATH)

current_col = 1  # Start in column A

for sheet in xlsx.sheet_names:

    # Write sheet name in row 1
    ws.cell(row=1, column=current_col, value=sheet)

    # Load the sheet to get the columns
    df = pd.read_excel(EXCEL_PATH, sheet_name=sheet)

    # Write each column name below
    for row_i, col_name in enumerate(df.columns, start=2):
        ws.cell(row=row_i, column=current_col, value=col_name)

    # Move 3 columns right → 1 for data, 2 blank columns between blocks
    current_col += 3

# Save updated workbook (same file)
wb.save(EXCEL_PATH)

print(f"Sheet '{DICT_SHEET}' created and saved into {EXCEL_PATH}")


Sheet 'dict_columns' created and saved into data/stats_t1_fs/ses2/ses2_T1_data.xlsx
